# **Dynamic Mapping and Quality of Service Driven Re-Embedding in Virtualization**

# Code implementation: Siva Bonthada (212IS034)  & Durga Vinay Bonthu (212IS006)
# Course: Virtualization and Cloud Computing - CS728
# Semester: II-M.Tech

In [2]:
!unzip alib.zip

'unzip' is not recognized as an internal or external command,
operable program or batch file.


#Installing the requirements

In [120]:
!pip install gurobipy
!pip install Unidecode

# Reading the Graph input from the pickle file

In [121]:
#readpickle.py
import pickle
with open('input.pickle', 'rb') as handle:
    b = pickle.load(handle)

sn_graph=b.get("substrate")
nodes_sn_graph=sn_graph.nodes
#Pringing the total number of nodes
print("Total number of nodes in the entire graph is : {}\n".format(nodes_sn_graph))

SN_node_CRB=sn_graph.node_weights
SN_edge_BW=sn_graph.edge_weights
SN_edge_Delay = sn_graph.delay

#Print nodes with weights assigned
print("The node weights are assigned as follows:\n{}\n".format(SN_node_CRB))
#Print edges weights which is bandwidth requirement of the link.
print("The edge weights / bandwidth are assigned as follows:\n{}\n".format(SN_edge_BW))
#Print edge propagation delay of each link.
print("The edge dealys are assigned as follows:\n{}\n".format(SN_edge_Delay))

Total number of nodes in the entire graph is : 84

The node weights are assigned as follows:
{0: 20384, 1: 127021, 2: 297374, 3: 140702, 4: 142586, 5: 359681, 6: 497295, 7: 233929, 8: 241931, 9: 159035, 10: 19151, 11: 466085, 12: 262973, 13: 432188, 14: 138003, 15: 375071, 16: 252534, 17: 322366, 18: 31042, 19: 468912, 20: 482582, 21: 78815, 22: 491010, 23: 218352, 24: 384703, 25: 98538, 26: 287869, 27: 104521, 28: 136565, 29: 283402, 30: 172827, 31: 336842, 32: 275990, 33: 436229, 34: 170474, 35: 72262, 36: 74810, 37: 382571, 38: 158420, 39: 354635, 40: 179284, 41: 141108, 42: 81267, 43: 373266, 44: 29542, 45: 266320, 46: 349271, 47: 40954, 48: 65226, 49: 464017, 50: 102288, 51: 483997, 52: 351462, 53: 416087, 54: 92170, 55: 53122, 56: 30219, 57: 176897, 58: 224640, 59: 434396, 60: 224371, 61: 74570, 62: 202438, 63: 55437, 64: 187814, 65: 203546, 66: 220189, 67: 105240, 68: 452043, 69: 165759, 70: 179369, 71: 143018, 72: 264898, 73: 494291, 74: 174341, 75: 293951, 76: 433841, 77: 4289

# Main function and create VNE function



In [122]:
#create vne
import networkx as nx
import random
import graph
from graph import Parameters
import numpy as np


def create_vne(min_nodes=2, max_nodes=10, no_requests=1, probability=0.4):
    random_node_list_arr = np.random.uniform(min_nodes, max_nodes, no_requests)
    random_node_list = [int(i) for i in random_node_list_arr]
    new_vne_req = []
    for req in random_node_list:
        G = nx.erdos_renyi_graph(req, probability, directed=False)
        ng = nx.to_dict_of_lists(G)
        g = {}
        for i in ng:
            g[i + 1] = []
            for j in ng[i]:
                g[i + 1].append(j + 1)

        if not nx.is_connected(G):
            null_node_list = [key for key, val in g.items() if not val]
            graph_node_count = {_key: len(_val) for _key, _val in g.items()}
            sorted_dict_list = sorted(
                graph_node_count.items(), key=lambda x: x[1], reverse=True
            )
            if len(null_node_list) != len(g):
                for index, empty_node in enumerate(null_node_list):
                    g[sorted_dict_list[index][0]].append(empty_node)
                    g[empty_node].append(sorted_dict_list[index][0])
            else:
                for i in range(len(g)):
                    for j in range(len(g) - i - 1):
                        if null_node_list[j + 1] not in g[null_node_list[j]]:
                            g[null_node_list[j]].append(null_node_list[j + 1])
                        if null_node_list[j] not in g[null_node_list[j + 1]]:
                            g[null_node_list[j + 1]].append(null_node_list[j])
        new_vne_req.append(g)

    print("Randomly produced new VNE REQ is : \n {} \n".format(new_vne_req))
    vne = []
    for i in range(len(new_vne_req)):
        edges = set()
        nodes = len(new_vne_req[i])
        for j in range(nodes):
            for k in new_vne_req[i][j + 1]:
                edges.add((str(j), str(k - 1)))
        vne.append(graph.Graph(nodes, edges, Parameters(1, 10, 1, 10, 0, 100, 0, 100, 1, 4)) )  # for vne request BW ,CRB, Location,Delay

    return vne


if __name__ == "__main__":
    my_vne=create_vne()

Randomly produced new VNE REQ is : 
 [{1: [2], 2: [3, 1], 3: [2]}] 



In [123]:
#Printing the edge weights of the newly created random VNE
print("The edge weights of the  newly created random VNE : \n{}\n".format(my_vne[0].edge_weights))

The edge weights of the  newly created random VNE : 
{('1', '0'): 8, ('0', '1'): 8, ('2', '1'): 4, ('1', '2'): 4}



# Sorting PN Nodes

In [124]:
#Sorting the nodes based on node weights.
sorted_values = sorted(SN_node_CRB.values(),reverse = True) # Sort the values
#Decalring an empty dictionary to store the sorted nodes and corresponding weights
sorted_PN = {}

for i in sorted_values:
    for k in SN_node_CRB.keys():
        if SN_node_CRB[k] == i:
            sorted_PN[k] = SN_node_CRB[k]
            break
print("Nodes in PN are sorted using node weights : \n{}\n".format(sorted_PN))

Nodes in PN are sorted using node weights : 
{6: 497295, 73: 494291, 22: 491010, 51: 483997, 20: 482582, 19: 468912, 11: 466085, 49: 464017, 68: 452043, 33: 436229, 59: 434396, 76: 433841, 13: 432188, 77: 428990, 53: 416087, 83: 398323, 24: 384703, 37: 382571, 15: 375071, 43: 373266, 5: 359681, 39: 354635, 52: 351462, 46: 349271, 80: 341075, 31: 336842, 17: 322366, 2: 297374, 75: 293951, 26: 287869, 29: 283402, 32: 275990, 45: 266320, 72: 264898, 12: 262973, 16: 252534, 8: 241931, 7: 233929, 58: 224640, 60: 224371, 66: 220189, 23: 218352, 65: 203546, 62: 202438, 64: 187814, 70: 179369, 40: 179284, 57: 176897, 74: 174341, 30: 172827, 34: 170474, 69: 165759, 9: 159035, 38: 158420, 71: 143018, 4: 142586, 41: 141108, 3: 140702, 14: 138003, 28: 136565, 1: 127021, 82: 119812, 67: 105240, 27: 104521, 50: 102288, 25: 98538, 54: 92170, 78: 89543, 42: 81267, 21: 78815, 36: 74810, 61: 74570, 35: 72262, 48: 65226, 63: 55437, 55: 53122, 47: 40954, 18: 31042, 56: 30219, 44: 29542, 0: 20384, 10: 1915

# Sorting VN Nodes

In [125]:
#Assigning the random weights to the nodes in the VNE.
import random
for w in my_vne[0].node_weights.keys():
  my_vne[0].node_weights[w] = random.randint(10000,99999)
print("Nodes in VN are assigned weights : \n{}\n".format(my_vne[0].node_weights))


Nodes in VN are assigned weights : 
{0: 63623, 1: 88575, 2: 12951}



In [126]:
#Sorting the nodes based on node weights.
sorted_values = sorted(my_vne[0].node_weights.values(),reverse = True) # Sort the values
#Decalring an empty dictionary to store the sorted nodes and corresponding weights
sorted_VN = {}

for i in sorted_values:
    for k in my_vne[0].node_weights.keys():
        if my_vne[0].node_weights[k] == i:
            sorted_VN[k] = my_vne[0].node_weights[k]
print("Nodes in VN are sorted using node weights : \n{}\n".format(sorted_VN))

Nodes in VN are sorted using node weights : 
{1: 88575, 0: 63623, 2: 12951}



In [127]:
print("Printing both PN and VN node weights after sorting out both : \nPN : {}\nVN : {}\n".format(sorted_PN,sorted_VN))

Printing both PN and VN node weights after sorting out both : 
PN : {6: 497295, 73: 494291, 22: 491010, 51: 483997, 20: 482582, 19: 468912, 11: 466085, 49: 464017, 68: 452043, 33: 436229, 59: 434396, 76: 433841, 13: 432188, 77: 428990, 53: 416087, 83: 398323, 24: 384703, 37: 382571, 15: 375071, 43: 373266, 5: 359681, 39: 354635, 52: 351462, 46: 349271, 80: 341075, 31: 336842, 17: 322366, 2: 297374, 75: 293951, 26: 287869, 29: 283402, 32: 275990, 45: 266320, 72: 264898, 12: 262973, 16: 252534, 8: 241931, 7: 233929, 58: 224640, 60: 224371, 66: 220189, 23: 218352, 65: 203546, 62: 202438, 64: 187814, 70: 179369, 40: 179284, 57: 176897, 74: 174341, 30: 172827, 34: 170474, 69: 165759, 9: 159035, 38: 158420, 71: 143018, 4: 142586, 41: 141108, 3: 140702, 14: 138003, 28: 136565, 1: 127021, 82: 119812, 67: 105240, 27: 104521, 50: 102288, 25: 98538, 54: 92170, 78: 89543, 42: 81267, 21: 78815, 36: 74810, 61: 74570, 35: 72262, 48: 65226, 63: 55437, 55: 53122, 47: 40954, 18: 31042, 56: 30219, 44: 29

# Initial Embedding

In [128]:
print("Updating the weights while assigning the VNs to PNs: \n")
assignment = {}
PN_status = [0] * 84
for i in sorted_VN.keys():
  for j in sorted_PN.keys():
    if(sorted_VN[i] <=  sorted_PN[j]):
      if(PN_status[j] == 0):
        assignment[i] = j
        PN_status[j] = 1
        sorted_PN[j] = (sorted_PN[j] - sorted_VN[i])
        print("The weight of PN {} after allocation and updation will be {} - {} = {} " .format(j, (sorted_PN[j] + sorted_VN[i]), sorted_VN[i] ,  sorted_PN[j] ))
        break
print("\nThe assignment of VN to PN will be: \n")
for v in assignment:
  print('VNode {} will be allocate to PNode {}.'.format(v,assignment[v]) )

print("\nThe updated node weights for PN will be : \n")
print(sorted_PN)

Updating the weights while assigning the VNs to PNs: 

The weight of PN 6 after allocation and updation will be 497295 - 88575 = 408720 
The weight of PN 73 after allocation and updation will be 494291 - 63623 = 430668 
The weight of PN 22 after allocation and updation will be 491010 - 12951 = 478059 

The assignment of VN to PN will be: 

VNode 1 will be allocate to PNode 6.
VNode 0 will be allocate to PNode 73.
VNode 2 will be allocate to PNode 22.

The updated node weights for PN will be : 

{6: 408720, 73: 430668, 22: 478059, 51: 483997, 20: 482582, 19: 468912, 11: 466085, 49: 464017, 68: 452043, 33: 436229, 59: 434396, 76: 433841, 13: 432188, 77: 428990, 53: 416087, 83: 398323, 24: 384703, 37: 382571, 15: 375071, 43: 373266, 5: 359681, 39: 354635, 52: 351462, 46: 349271, 80: 341075, 31: 336842, 17: 322366, 2: 297374, 75: 293951, 26: 287869, 29: 283402, 32: 275990, 45: 266320, 72: 264898, 12: 262973, 16: 252534, 8: 241931, 7: 233929, 58: 224640, 60: 224371, 66: 220189, 23: 218352, 

# Checking the link delays

In [129]:
print("PN link delays are as follows : \n{}\n".format(SN_edge_Delay))

PN link delays are as follows : 
{('22', '18'): 1, ('18', '22'): 1, ('3', '2'): 1, ('2', '3'): 1, ('23', '31'): 1, ('31', '23'): 1, ('19', '38'): 1, ('38', '19'): 1, ('17', '49'): 1, ('49', '17'): 1, ('9', '10'): 1, ('10', '9'): 1, ('61', '21'): 1, ('21', '61'): 1, ('28', '26'): 1, ('26', '28'): 1, ('70', '50'): 1, ('50', '70'): 1, ('37', '60'): 1, ('60', '37'): 1, ('82', '34'): 1, ('34', '82'): 1, ('71', '6'): 1, ('6', '71'): 1, ('67', '51'): 1, ('51', '67'): 1, ('27', '5'): 1, ('5', '27'): 1, ('45', '30'): 1, ('30', '45'): 1, ('20', '22'): 1, ('22', '20'): 1, ('77', '78'): 1, ('78', '77'): 1, ('48', '54'): 1, ('54', '48'): 1, ('79', '34'): 1, ('34', '79'): 1, ('33', '60'): 1, ('60', '33'): 1, ('80', '36'): 1, ('36', '80'): 1, ('6', '54'): 1, ('54', '6'): 1, ('45', '1'): 1, ('1', '45'): 1, ('4', '10'): 1, ('10', '4'): 1, ('68', '69'): 1, ('69', '68'): 1, ('61', '69'): 1, ('69', '61'): 1, ('55', '11'): 1, ('11', '55'): 1, ('42', '41'): 1, ('41', '42'): 1, ('11', '60'): 1, ('60', '11'):

In [130]:
#print(my_vne[0].delay)
#Printing VN delays
import random
for w in my_vne[0].delay.keys():
  my_vne[0].delay[w] = random.randint(5,20)
print(my_vne[0].delay)

{('1', '0'): 14, ('0', '1'): 9, ('2', '1'): 18, ('1', '2'): 19}


In [131]:
print(assignment)

{1: 6, 0: 73, 2: 22}


# Finding QoS criteria of our Embedding

In [132]:
for i in my_vne[0].delay.keys():
  #print("For {} The assignments are : " .format(i))
  for j in assignment.keys():
    if(j == int(i[0])):
      first_node = str(assignment[j])
    elif(j == int(i[1])):
      second_node = str(assignment[j])
  #Getting delay of this particular link in VN
  dly = my_vne[0].delay[i]
  print("Shortest delay path in PN : First node : {} and Second node : {}".format(first_node,second_node))
  #Finding the shortest path which has minimum delay.
  pat = sn_graph.findShortestPath(first_node,second_node,dly)
  #Printing the path that has been found by the algorithm.
  print("The path for the above path is: {}".format(pat))
  #Computing the same link delay in the PN using shortest link dealy algorithm
  res_del = 0
  for m in range(0,len(pat)-1):
    res_del = res_del + SN_edge_Delay[(str(pat[m]),str(pat[m+1]))]
  #Printing the path delay in PN
  print("Path delay is : {}".format(res_del))
  #Comparing the PN and VN delays to check whether QoS requirements met or not.
  if(dly >=  res_del):
    print("This link satisifies the requirements!!")
  else:
    print("This link does not satisify the requirements. So we need the reembedd!")

  

Shortest delay path in PN : First node : 6 and Second node : 73
The path for the above path is: ['6', '54', '48', '20', '22', '18', '21', '61', '69', '46', '47', '78', '74', '73']
Path delay is : 13
This link satisifies the requirements!!
Shortest delay path in PN : First node : 73 and Second node : 6
The path for the above path is: ['73', '74', '78', '47', '46', '69', '61', '21', '18', '22', '20', '48', '54', '6']
Path delay is : 13
This link does not satisify the requirements. So we need the reembedd!
Shortest delay path in PN : First node : 22 and Second node : 6
The path for the above path is: ['22', '20', '48', '54', '6']
Path delay is : 4
This link satisifies the requirements!!
Shortest delay path in PN : First node : 6 and Second node : 22
The path for the above path is: ['6', '54', '48', '20', '22']
Path delay is : 4
This link satisifies the requirements!!


# Dynamic Embedding for the unsatisified link end nodes.

The one with the less node weight will be re-embedded.